# HW4

In [1]:
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
from datasets import load_dataset
from datasets.arrow_dataset import Dataset
from datasets.dataset_dict import DatasetDict, IterableDatasetDict
from datasets.iterable_dataset import IterableDataset
import numpy as np

# Dataset id from huggingface.co/dataset
# dataset_id = "argilla/synthetic-domain-text-classification"
dataset_id = "sentence-transformers/all-nli"

# Load raw dataset
# train_dataset = load_dataset(dataset_id, split='train')
train_dataset = load_dataset(dataset_id, 'pair-class', split='train[:3%]')
#divide train set into fraction of the size, so more computationally feasible


split_dataset = train_dataset.train_test_split(test_size=0.1)
# print(split_dataset['train'][5:7])
print(split_dataset)

# create a new column in split_dataset which concatenates the "premise" and "hypothesis" columns
split_dataset = split_dataset.map(lambda x: {"text": x["premise"] + " <s> " + x["hypothesis"]}, remove_columns=["premise", "hypothesis"])
# print(split_dataset['train'][5:7])
# print(split_dataset)

from transformers import AutoTokenizer

# Model id to load the tokenizer
model_id = "answerdotai/ModernBERT-base"

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Tokenize helper function
def tokenize(batch):
    return tokenizer(batch['text'], truncation=True, padding="max_length", return_tensors="pt", max_length=500)#tokenizer.model_max_length)


# Tokenize dataset
if "label" in split_dataset["train"].features.keys():
    split_dataset =  split_dataset.rename_column("label", "labels") # to match Trainer
tokenized_dataset = split_dataset.map(tokenize, batched=True, remove_columns=["text"])

from transformers import AutoModelForSequenceClassification

# Model id to load the tokenizer
model_id = "answerdotai/ModernBERT-base"

# Prepare model labels - useful for inference
labels = tokenized_dataset["train"].features["labels"].names
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

# Download the model from huggingface.co/models
model = AutoModelForSequenceClassification.from_pretrained(
    model_id, num_labels=num_labels, label2id=label2id, id2label=id2label,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


train-00000-of-00001.parquet:   0%|          | 0.00/69.5M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

Map:   0%|          | 0/25435 [00:00<?, ? examples/s]

Map:   0%|          | 0/2827 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import Trainer, TrainingArguments

# Define training args
training_args = TrainingArguments(
    output_dir= "ModernBERT-domain-classifier",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    learning_rate=5e-5,
    num_train_epochs=2,
    bf16=True, # bfloat16 training
    optim="adamw_torch_fused", # improved optimizer
    # logging & evaluation strategies
    logging_strategy="steps",
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_total_limit=2,
    load_best_model_at_end=True,
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss,Validation Loss
500,0.941100,0.633008
1000,0.738000,0.712582
1500,0.715700,0.619194
2000,0.643000,0.619654
2500,0.671700,0.616809
3000,0.627600,0.562970
3500,0.588400,0.562796
4000,0.605700,0.468529
4500,0.617300,0.503838
5000,0.574000,0.523241


TrainOutput(global_step=12718, training_loss=0.4974766830393945, metrics={'train_runtime': 2336.685, 'train_samples_per_second': 21.77, 'train_steps_per_second': 5.443, 'total_flos': 1.692819511767e+16, 'train_loss': 0.4974766830393945, 'epoch': 2.0})

In [6]:
print("Sample training data:")
label_names = tokenized_dataset["train"].features["labels"].names
for i in range(5):
    example = split_dataset["train"][i]
    print("\nExample Input:", example["text"])
    print("Actual Label:", label_names[example["labels"]])

Sample training data:

Example Input: a motorcyclist does a nose wheelie. <s> A man standing beside a crashed motorcycle
Actual Label: contradiction

Example Input: A surfer on a blue surfboard is falling off of it as he hits a wave. <s> A shark is surfing.
Actual Label: contradiction

Example Input: A yellow motorcycle drives down the road. <s> A cyclist driving a yellow motorcycle.
Actual Label: entailment

Example Input: A man standing on a motorcycle doing a stunt <s> a man crashes a car into a pole
Actual Label: contradiction

Example Input: A girl is wearing a blue wig under a brown hooded jacket. <s> A girl's blue wig is visible under her brown-hooded jacket.
Actual Label: entailment


In [ ]:
import numpy as np

predictions = trainer.predict(tokenized_dataset["test"])

# Process the prediction results (predictions, label_ids, metrics)
predicted_labels = np.argmax(predictions.predictions, axis=1)
predicted_label = id2label[str(predicted_labels[0])]

example_data = [split_dataset['train'][0]]
print("Example Input:", example_data[0]['text'])
print(f"Predicted Label: {predicted_label}")
print(f"Actual Label: {id2label[str(example_data[0]['labels'])]}")


Example Input: a motorcyclist does a nose wheelie. <s> A man standing beside a crashed motorcycle
Predicted Label: entailment
Actual Label: contradiction


### Question 1
Analyze the true positive / false positive / true negative / false negative rates overall, treating every answer as either wrong or right (e.g., if the correct answer was 0, then only 0 is correct and 1 or 2 are both wrong). What results do you get? What does this tell you?

-------

In this part of the project, we retrieved the statistics from our trained and tested ModernBERT model. There are three indices and three corresponding labels: [0: "entailment", 1: "neutral", 2: "contradiction"]. It is important to mention that in this question, we treated every answer as either wrong or right. This means that predicted answers could be separated only into two categories: true positives and false negatives. That's why in this setup, false positives and true negatives were initialized to zero. The model correctly predicted labels in 2827 test cases, which were classified as true positives. However, 465 predictions were incorrect, making them false negatives. The total number of samples is 2,827.

The total accuracy was also calculated, which is simply the rate of true positives over the total number of samples. According to the statistics, it is 0.8355 or 83.55%. While this represents relatively good performance, it also means that 16.45% of the time, the model fails to predict the correct label. These errors could arise from ambiguous inputs or class imbalance. For example, from the previous test example we have: "Example Input: a motorcyclist does a nose wheelie. \<s> A man standing beside a crashed motorcycle. Predicted Label: entailment. Actual Label: contradiction." This shows why the model does not achieve 100% performance. One reason is that the model may be biased toward predicting "entailment" or "neutral" if the example input is too ambiguous. However, this is one of the limitations of our current setup (right or wrong). We still need to understand in which specific classes our model makes mistakes more often. This will help us get the full picture and complete our analysis.

In [8]:
#Retrieve actual and predicted indexes
actual = predictions.label_ids
predicted = np.argmax(predictions.predictions, axis=1) #Take the argmax to get predicted labels
#Set variables
true_positive = np.sum(actual == predicted) #In true positives, labels match
false_positive = 0 #We treat every answer as either wrong or right
true_negative = 0
false_negative = np.sum(actual != predicted) #In false negatives, labels don't match
total_accuracy = true_positive / len(actual) #Calculate the total accuracy
#Print results
print(f"Total samples: {len(actual)}")
print(f"True positives: {true_positive}")
print(f"False positives: {false_positive}")
print(f"True negatives: {true_negative}")
print(f"False negatives: {false_negative}")
print(f"Accuracy: {total_accuracy:.4f}")

Total samples: 2827
True positives: 2362
False positives: 0
True negatives: 0
False negatives: 465
Accuracy: 0.8355


### Question 2
Now analyze the TP/FP/TN/FN rates *for each category* (contradiction, entailment, neutral). What do you learn? What does this tell you that you couldn't tell from the answer to Question 1?

-------

According to the obtained classification report, we have the necessary statistics for each category: entailment, neutral, and contradiction. In the entailment category, the model predicted 838 true positives, 1,722 true negatives, 169 false positives, and 98 false negatives. This means that in most cases, the model correctly assigns the prediction. However, the model can still wrongly assign the entailment label to other cases (false positives), which shows that it sometimes mistakes neutral or contradiction examples as entailment. In the neutral category, the model predicted 749 true positives, 1,678 true negatives, 195 false positives, and 205 false negatives. We can observe that while the model has a lot of true positives, it often either wrongly assigns this category to other cases (false positives) or just misses neutral statements completely (false negatives). This tells us that neutral examples are harder for the model to recognize accurately. In the contradiction category, the model predicted 775 true positives, 1,789 true negatives, 101 false positives, and 162 false negatives. While in this case the model still has a number of false negatives, it has the least number of false positives compared to the other categories.

From this data, we can see that the most challenging category for our model is neutral. It has the highest number of false positives and false negatives out of all three categories. Moreover, the precision rate for neutral is also the lowest at 0.79, while the contradiction class has the highest precision at 0.88. This shows that the model is much more accurate when predicting contradiction than neutral. Neutral statements are harder to predict because they often sound very similar to either entailment or contradiction depending on how the sentences are worded. Thus, there is still room for improvement, especially in neutral category.

We learned a lot more from this analysis than from Question 1 because now we can actually see where and how often the model makes mistakes for each specific category. Before, we could only tell whether a prediction was right or wrong overall. Now, we know that neutral examples are the biggest problem, and we can focus on improving this area. For example, we can add clearer neutral examples into a training dataset.

In [9]:
from sklearn.metrics import classification_report
import pandas as pd
print(classification_report(actual, predicted, target_names=labels)) #Print the classification report for our model
data = pd.DataFrame({'actual': actual, 'predicted': predicted}) #Combine both actual and predicted labels into a single DataFrame
for index, label in enumerate(labels): #Interate over each category using label names and their indexes
    true_positive = ((data.actual == index) & (data.predicted == index)).sum() #The model correctly predicted this category
    false_positive = ((data.actual != index) & (data.predicted == index)).sum() #The model predicted this category, but the actual category was different
    true_negative = ((data.actual != index) & (data.predicted != index)).sum() #The model correctly predicted another category
    false_negative = ((data.actual == index) & (data.predicted != index)).sum() #The actual label is this category, but the model predicted something else
    print(f"{label:19s} TP={true_positive:3d}, FP={false_positive:3d}, TN={true_negative:3d}, FN={false_negative:3d}") #Print calculated values

               precision    recall  f1-score   support

   entailment       0.83      0.90      0.86       936
      neutral       0.79      0.79      0.79       954
contradiction       0.88      0.83      0.85       937

     accuracy                           0.84      2827
    macro avg       0.84      0.84      0.84      2827
 weighted avg       0.84      0.84      0.84      2827

entailment          TP=838, FP=169, TN=1722, FN= 98
neutral             TP=749, FP=195, TN=1678, FN=205
contradiction       TP=775, FP=101, TN=1789, FN=162


### Question 3
Analyze specific errors. For example, look at questions that had (x) as a correct answer but the model guessed (y) instead. Look at specific cases. What do you notice? Come up with at least 4 hypotheses about what kinds of problems it's getting right and wrong.

-------

Now, we are going to look at specific cases where the model made a mistake. For example, at index 464, we have the sentence: "Workers are at work under scaffolding. \<s> Working under extreme conditions." The actual label is neutral, however, the model predicted it to be entailment. The hypothesis has the words "extreme conditions," which add a different meaning. However, the model still fails to recognize this change. It can be assumed that the model focuses on the matching topic while not performing a deeper analysis.

In the next example, at index 1348, we have the sentence: "One man in a blue shirt is operating a watering hose and another man in a brown shirt is moving tarps. \<s> Two men are setting up a swimming pool." The actual label is neutral, however, the model predicted it to be a contradiction. In the hypothesis, the input data just adds more information about the specific activity. However, the model treats it as a contradiction even though there is no implied information that is contradicting.

Next, we have: "A group of children run a footrace in the snow. \<s> The children are playing with each other." The actual label is neutral, however, the model predicted it to be entailment. The model assumed that if the activity is done by children, then they are always playing. However, this is not true. We can see that the model is overgeneralizing and making basic assumptions.

As I analyzed all other examples, I came up with four hypotheses about why the model might fail:

Hypothesis 1: The model mistakes similar topics as entailment and does not focus on exact meaning.

Hypothesis 2: It struggles with abstract meaning connections.

Hypothesis 3: The model often overgeneralizes specific activities into broader categories.

Hypothesis 4: It makes assumptions based on the most likely possibilities that may not be valid.

In [12]:
import pandas as pd
text = split_dataset["test"] #Untokenized data set with premises and hypotheses
data1 = pd.DataFrame({ #DataFrame with combined input data, actual, and predicted labels
    "text": text["text"],
    "actual_label": [label_names[i] for i in actual], #Convert actual label indexes to names
    "predicted_label": [label_names[i] for i in predicted]}) #Convert predicted label indexes to names
errors = data1[data1.actual_label != data1.predicted_label] #Retrieve rows where actual label does not match the predicted one
errors.sample(10, random_state=42) #Random sample

,text,actual_label,predicted_label
357,A kid walks on a snowy hill with their arms ou...,entailment,neutral
464,Workers are at work under scaffolding. <s> Wor...,neutral,entailment
205,Person in blue jacket on a bike on a dirt cour...,entailment,neutral
2787,A group of children run a footrace in the snow...,neutral,entailment
1758,A man with a hat and headset holding a green r...,neutral,contradiction
1348,One man in a blue shirt is operating a waterin...,neutral,contradiction
1920,A man dressed as a pirate stands next to a man...,neutral,contradiction
56,people enjoying their meals at an expensive re...,neutral,contradiction
2762,A group of Japanese men in bright red shirts p...,neutral,contradiction
1437,"Two woman playing a frisbee game, the frisbee ...",contradiction,entailment


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('actual_label').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('predicted_label').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['predicted_label'].value_counts()
    for x_label, grp in _df_4.groupby('actual_label')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('actual_label')
_ = plt.ylabel('predicted_label')

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_5['actual_label'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_5, x='index', y='actual_label', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_6['predicted_label'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_6, x='index', y='predicted_label', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

### Question 4

Given your hypotheses from question 3, create a few test cases manually and see if they confirm or refute your hypotheses.

-------

Now, we are going to see if the test cases we made confirm or go against the hypotheses from Question 3. In the first case, "A woman is slicing a pie in the kitchen. \<s> A woman is baking a pie in the kitchen" the model predicted entailment instead of neutral. This supports Hypothesis 1, because the model confused two similar activities and didn't focus on the real difference. It saw that both sentences talked about a pie in the kitchen and thought they must be directly connected, even though the actions are different.

In the second case, "He is angry. \<s> He is not calm" the model predicted contradiction instead of entailment. This supports Hypothesis 2, because the model struggles with abstract meaning. It treated being angry and not being calm like complete opposites, instead of seeing the natural connection between them.

In the third case, "They are throwing a ball to each other. \<s> They are playing" the model predicted entailment, and the actual label was also entailment. This supports Hypothesis 3 in a good way. The model correctly figured out that throwing a ball is a kind of playing. But in other examples, like kids running a race, it overgeneralized and made a mistake.

In the last case, "The weather is sunny right now. \<s> She will not take an umbrella today" the model predicted contradiction, but the actual label was neutral. This supports Hypothesis 4, because the model made an assumption based on what is most likely, even though it is not guaranteed.

Overall, the examples we tested mostly confirm the four hypotheses we made about the model’s behavior.

In [39]:
from transformers import pipeline
examples = [
    {"premise": "A woman is slicing a pie in the kitchen.", #Hypothesis 1
     "hypothesis": "A woman is baking a pie in the kitchen.",
     "actual": "neutral"},
    {"premise": "He is angry.", #Hypothesis 2
     "hypothesis": "He is not calm.",
     "actual": "entailment"},
    {"premise": "They are throwing a ball to each other.", #Hypothesis 3
     "hypothesis": "They are playing.",
     "actual": "entailment"},
    {"premise": "The weather is sunny right now.", #Hypothesis 4
     "hypothesis": "She will not take an umbrella today.",
     "actual": "neutral"}]
model.config.id2label = { i: name.upper()
                          for i, name in enumerate(label_names) }
model.config.label2id = { name.upper(): i
                          for i, name in enumerate(label_names) }
#Create a text-classification pipeline
classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    device=0,
    top_k=1
)
#Iterate over each example
for example in examples:
    text = example["premise"] + " <s> " + example["hypothesis"] #Combine into one string
    label = classifier(text)[0][0] #Retrieve the predicted label
    predicted = label["label"].lower()#Turn to lowercase
    #Print results
    print(f"\nPremise: {example['premise']}")
    print(f"Hypothesis: {example['hypothesis']}")
    print(f"Predicted: {predicted}")
    print(f"Actual: {example['actual']}")

Device set to use cuda:0



Premise: A woman is slicing a pie in the kitchen.
Hypothesis: A woman is baking a pie in the kitchen.
Predicted: entailment
Actual: neutral

Premise: He is angry.
Hypothesis: He is not calm.
Predicted: contradiction
Actual: entailment

Premise: They are throwing a ball to each other.
Hypothesis: They are playing.
Predicted: entailment
Actual: entailment

Premise: The weather is sunny right now.
Hypothesis: She will not take an umbrella today.
Predicted: contradiction
Actual: neutral
